### PYTHON LIBRARY SETUP

In [ ]:
import kagglehub #import module
import pandas as pd #import module
import os #import module
import re #import module

### DOWNLOAD AND LOAD DATASET

In [ ]:
# Download Dataset
path_fake_real = kagglehub.dataset_download(
    "clmentbisaillon/fake-and-real-news-dataset"
)
print("fake/real path:", path_fake_real)

path_ai1 = kagglehub.dataset_download(
    "walidbenaouda/ai-isot-dataset"
)
path_ai2 = kagglehub.dataset_download(
    "atharvasoundankar/gen-ai-misinformation-detection-datase-20242025"
)

In [ ]:
# Load the Fake and True CSV Datasets
fake = pd.read_csv(os.path.join(path_fake_real, "Fake.csv")) #cvs for fake
true = pd.read_csv(os.path.join(path_fake_real, "True.csv")) #cvs for true

# Load AI Datasets
ai_isot = pd.read_csv(os.path.join(path_ai1, "AI-ISOT dataset.csv"))
ai_gen = pd.read_csv(os.path.join(path_ai2, "generative_ai_misinformation_dataset.csv"))

In [ ]:
print(f"-----------------------FAKE.CSV---------------------{fake.head()}\n") #shows headers
print(f"-----------------------TRUE.CSV---------------------{true.head()}\n") #shows headers
print(f"-----------------------AI_1.CSV----------------------{ai_isot.head()}\n")   #shows headers
print(f"-----------------------AI_2.CSV----------------------{ai_gen.head()}\n")   #shows headers

### TEXT CLEANING

#### Data Cleaning Function

In [ ]:
# Cleaning text in the dataset
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www.\S+", "", text) # Gets rid of urls
    text = re.sub(r"<.*?>", "", text) # Gets rid of html
    text = re.sub(r"[^a-zA-Z0-9.,!?'’\s]", " ", text) # Makes sure punctuation still there
    text = re.sub(r"\s+", " ", text).strip() # Fixes spaces
    return text

#### Clean Fake vs True Dataset

In [ ]:
# Clean real vs fake dataset
fake["is_real"] = 0 # Gives fake news a label
true["is_real"] = 1 # Gives true news a label

df_rf = pd.concat([fake, true], ignore_index=True) # Combines datasets

# Makes text in one place
df_rf["text"] = (
    df_rf["title"].fillna("") + " " +
    df_rf["text"].fillna("")
).str.strip()

# Gets rid of duplicates and empty text
df_rf = df_rf.drop_duplicates(subset=["text"])
df_rf = df_rf.dropna(subset=["text"])
df_rf = df_rf[df_rf["text"].str.strip() != ""]
# Cleans the text
df_rf["text"] = df_rf["text"].apply(clean_text)

# Picks the necessary columns and saves the cleaned file
df_rf_clean = df_rf[["text", "is_real"]]
df_rf_clean["source"] = "FAKE-REAL"
df_rf_clean.to_csv("clean_real_fake.csv", index=False)

# Show it was saved and shows portion of cleaned data set
print("saved: clean_real_fake.csv")
print(df_rf_clean.head())

#### Clean AI vs Human Dataset

##### AI-Dataset 1 - AI-ISOT Dataset

In [ ]:
print(ai_isot.columns)

In [ ]:
ai_isot_long = []

# Human-written REAL news : human, real
for x in ai_isot["Real News"].dropna():
    ai_isot_long.append({"text": x, "is_ai": 0, "is_real": 1})

# Human-written FAKE news : human, fake
for x in ai_isot["Fake News"].dropna():
    ai_isot_long.append({"text": x, "is_ai": 0, "is_real": 0})

# AI-generated Fake News : ai, fake
for x in ai_isot["AI-generated Fake News"].dropna():
    ai_isot_long.append({"text": x, "is_ai": 1, "is_real": 0})

ai_isot_df = pd.DataFrame(ai_isot_long)

ai_isot_df["text"] = ai_isot_df["text"].apply(clean_text)
ai_isot_df["source"] = "AI-ISOT"

ai_isot_df.to_csv("clean_ai_isot.csv", index=False)
print("Saved: clean_ai_isot.csv")

##### AI-Dataset 2 - AI_Gen Dataset

In [ ]:
print(ai_gen.columns)

In [ ]:
# Keep only needed columns
ai_gen = ai_gen[[
    "text", "is_misinformation", "model_signature",
    "date", "month", "country", "platform"
]]

# Drop rows with missing text
ai_gen = ai_gen.dropna(subset=["text"])
ai_gen = ai_gen[ai_gen["text"].str.strip() != ""]

# Clean text
ai_gen["text"] = ai_gen["text"].apply(clean_text)

# Convert model_signature to AI/Human label
# 1 = AI-generated, 0 = Human-written
ai_gen = ai_gen[ai_gen["model_signature"].isin(["GPT-like", "human"])]
ai_gen["is_ai"] = ai_gen["model_signature"].apply(
    lambda x: 1 if x == "GPT-like" else 0
)

# Convert misinformation column to binary label
ai_gen["is_real"] = ai_gen["is_misinformation"].apply(
    lambda x: 0 if x == 1 else 1
)

# Final cleaned AI-gen dataset
ai_gen_clean = ai_gen[[
    "text", "is_real", "is_ai", "date", "month", "country", "platform"
]]

ai_gen_clean.to_csv("clean_ai_gen.csv", index=False)
print("Saved: clean_ai_gen.csv")

#### Combined Master Datasets

In [ ]:
combined = pd.concat([df_rf_clean, ai_isot_df, ai_gen], ignore_index=True)

combined = combined.drop_duplicates(subset=["text"])
combined = combined[combined["text"].str.strip() != ""]

combined.to_csv("combined_master_dataset.csv", index=False)
print("Saved: combined_master_dataset.csv")

print("Rows in final dataset:", len(combined))